
##**ChipNet**

This notebook demonstrates the implementation of the paper - **ChipNet: Budget-Aware Pruning with Heaviside Continuous Approximations**

Steps to train a baseline model and then compress it for a given budget are as follows: 
*   Load the YAML file.
*   Load dataset and create dataloaders.
*   Create ChipNet object and pass the parameters in the form of a dictionary.
*   Pass the dataloaders into the compress_model method to obtain the compressed model.

Since this is a demo notebook the number of epochs have been set to 1, 1 and 2 respectively for pretraining, pruning and finetuning respectively.



In [2]:
cd /Users/ayushsharma/Documents/Programming/Python/nyun/trailmet/

/Users/ayushsharma/Documents/Programming/Python/nyun/trailmet


In [3]:
ls

LICENSE      data/        experiments/ trailmet/
README.md    docs/        logs/


In [7]:
import sys
print(sys.path)

['/Users/ayushsharma/Documents/Programming/Python/nyun/trailmet/experiments/pruning/chipnet', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python310.zip', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload', '', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages']


In [4]:
import torch
import matplotlib.pyplot as plt
from torchvision import transforms
from trailmet.models import ModelsFactory
from trailmet.datasets.classification import DatasetFactory
import yaml

In [8]:
# root = "/content/trailmet/experiments/pruning/chipnet"
root = "/Users/ayushsharma/Documents/Programming/Python/nyun/trailmet/experiments/pruning/chipnet"

###Loading the yaml file for the configurations.

In [9]:
import os
with open(os.path.join(root, "chipnet.yaml"), 'r') as stream:
    data_loaded = yaml.safe_load(stream)
print(data_loaded)

{'CHIPNET_ARGS': {'BUDGET_TYPE': 'channel_ratio', 'TARGET_BUDGET': 0.5, 'BUDGET_LOSS_WEIGHTAGE': 30, 'CRISPNESS_LOSS_WEIGHTAGE': 10, 'BETA_INCREMENT': 5, 'GAMMA_INCREMENT': 2, 'STEEPNESS': 100}, 'PRETRAIN': {'EPOCHS': 1, 'TEST_ONLY': False, 'OPTIMIZER': 'SGD', 'LR': 0.05, 'SCHEDULER_TYPE': 1, 'WEIGHT_DECAY': 0.001}, 'PRUNE': {'EPOCHS': 1, 'TEST_ONLY': False, 'OPTIMIZER': 'SGD', 'LR': 0.05, 'WEIGHT_DECAY': 0.001}, 'FINETUNE': {'EPOCHS': 2, 'TEST_ONLY': False, 'OPTIMIZER': 'SGD', 'LR': 0.05, 'SCHEDULER_TYPE': 1, 'WEIGHT_DECAY': 0.001}}


###Loading the model.

In [10]:
model = ModelsFactory.create_model('resnet50', 100, False, insize=32)

###Loading the dataset.

In [12]:
cd ..

/Users/ayushsharma/Documents/Programming/Python/nyun


In [13]:
ls

checkpoints/  data_dir/     logs/         torchtest.py  trailmet/


In [20]:
from trailmet.datasets.classification import DatasetFactory
data_root = "data_dir"

In [62]:
mkdir data_dir

In [21]:
data_root

'data_dir'

In [22]:
train_transform = transforms.Compose(
[transforms.ToTensor()])

val_transform = transforms.Compose(
[transforms.ToTensor()])

test_transform = transforms.Compose(
[transforms.ToTensor()])

transforms1 = {
    'train': train_transform, 
    'val': val_transform, 
    'test': test_transform}
def train_target_transform(label):
    return label

def val_target_transform(label):
    return label

def test_target_transform(label):
    return label

target_transforms = {
    'train': None, 
    'val': None, 
    'test': None}


cifar_dataset = DatasetFactory.create_dataset(name = 'CIFAR100', 
                                        root = data_root,
                                        split_types = ['train', 'val', 'test'],
                                        val_fraction = 0.2,
                                        transform = transforms1,
                                        target_transform = target_transforms
)


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [19]:
# mkdir /data_dir

mkdir: /data_dir: Read-only file system


###Creating the dataloaders.

In [23]:
train_loader = torch.utils.data.DataLoader(
        cifar_dataset['train'], batch_size=64, 
        sampler=cifar_dataset['train_sampler'],
        num_workers=0
    )
val_loader = torch.utils.data.DataLoader(
        cifar_dataset['val'], batch_size=64, 
        sampler=cifar_dataset['val_sampler'],
        num_workers=0
    )
test_loader = torch.utils.data.DataLoader(
        cifar_dataset['test'], batch_size=64, 
        sampler=cifar_dataset['test_sampler'],
        num_workers=0
    )

###Creating the method's object followed by compression

In [24]:
from trailmet.algorithms.prune.chipnet import ChipNet

In [25]:
a = ChipNet(model, {'train': train_loader, 'val': val_loader, 'test': test_loader}, **data_loaded)

In [26]:
a.compress_model()

100%|██████████| 157/157 [05:11<00:00,  1.98s/it, acc1=11.1, acc5=33.6, loss=3.78]


**Saving model**


100%|██████████| 157/157 [05:08<00:00,  1.96s/it, acc1=11.2, acc5=33.9, loss=3.77]


Test Accuracy: 11.19625796178344 | Valid Accuracy: 11.076831210191083
preparing model for pruning
Starting epoch 1 / 1


100%|██████████| 625/625 [31:58<00:00,  3.07s/it, loss=5.41]  


...

Changed beta to 1.02 changed gamma to 2.8284271247461903
**Saving checkpoint**


100%|██████████| 157/157 [05:25<00:00,  2.07s/it, acc1=3.1, acc5=13.1, loss=4.38] 


**Saving model**


100%|██████████| 157/157 [05:19<00:00,  2.03s/it, acc1=2.91, acc5=13.1, loss=4.39]

Test Accuracy: 2.906050955414013 | Valid Accuracy: 3.0951433121019107
